# PPT 5주차


### 4.4.2. 신경망에서의 기울기

In [11]:
import numpy as np
import pandas
from functions import *
from gradient import numerical_gradient

class simpleNet_2x3:
    def __init__(self):
        self.w = np.random.randn(2, 3)
    def predict(self, x):
        return np.dot(x, self.w)
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss=cross_entropy_error(y, t)
        return loss

In [12]:
net = simpleNet_2x3()
print(net.w)

[[-0.47029095 -0.0986776   0.33701501]
 [-1.02369413 -0.32536875 -0.0458128 ]]


In [13]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)
np.argmax(p)

[-1.20349929 -0.35203843  0.16097749]


2

In [14]:
t = np.array([1, 0, 0])
print(net.loss(x, t))

1.9819302478168852


In [15]:
def f(w):
    return net.loss(x, t)
dw = numerical_gradient(f, net.w)
print(dw)

[[-0.51731789  0.19372872  0.32358918]
 [-0.77597684  0.29059307  0.48538376]]


# PPT 6주차

### 4.5. 학습 알고리즘 구현

In [16]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        self.params = {}
        self.params['w1'] =  weight_init_std * \
            np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['w2'] = weight_init_std * \
            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    def predict(self, x):
        w1, w2 = self.params['w1'], self.params['w2']
        b1, b2 = self.params['b1'], self.params['b2']
        a1 = np.dot(x, w1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, w2) + b2
        y = softmax(a2)
        return y
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy
    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)
        grads = {}
        grads['w1'] = numerical_gradient(loss_w, self.params['w1'])
        grads['b1'] = numerical_gradient(loss_w, self.params['b1'])
        grads['w2'] = numerical_gradient(loss_w, self.params['w2'])
        grads['b2'] = numerical_gradient(loss_w, self.params['b2'])
        return grads
#출력 존나 안 나와서 확장할 거임
def gradient(self, x, t):
    w1, w2 = self.params['w1'], self.params['w2']
    b1, b2 = self.params['b1'], self.params['b2']
    grads = {}
    batch_num = x.shape[0]
    
    #forward
    a1 = np.dot(x, w1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, w2) +b2
    y = softmax(a2)
    
    #backward - 역산: 제대로 된 출력을 위한 가중치 조절
    dy = (y-t) / batch_num
    grads['w2'] = np.dot(z1.T, dy)
    grads['b2'] = np.sum(dy, axis=0)
    da1 = np.dot(dy, w2.T)
    dz1 = sigmoid_grad(a1) * da1
    grads['w1'] = np.dot(x.T, dz1)
    grads['b1'] = np.sum(dz1, axis=0)
    return grads

In [18]:
#위 신경망을 mnist에 적용
from datasetsub.mnist import load_mnist
(x_train, y_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)
train_loss_list = []
iters_sum = 100
learning_rate = 0.1
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
for i in range(iters_sum):
    grad = network.numerical_gradient(x_train, y_train)
    for key in ('w1', 'b1', 'w2', 'b2'):
        network.params[key] -= learning_rate*grad[key]
    loss = network.loss(x_train, y_train)
    train_loss_list.append(loss)


ModuleNotFoundError: No module named 'numpy._core.numeric'

### 4.5.1. 미니배치 학습 구현

In [ ]:
(x_train, y_train), (x_test, y_test) = \
    load_mnist(normalize=True, one_hot_label=True)
train_loss_list = []
iters_sum = 10000   #10000번 말고 다른 숫자로 수정
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_sum):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    y_batch = y_train[batch_mask]
    grad = network.numerical_gradient(x_batch, y_batch)
    for key in ('w1', 'b1', 'w2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    loss = network.loss(x_batch, y_batch)
    train_loss_list.append(loss)
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, y_train)
        test_acc = network.accuracy(x_test, y_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(train_acc)
        print('훈련정확도, 검증정확도: ' + str(train_acc), + ', ' + str(test_acc))

ModuleNotFoundError: No module named 'numpy._core.numeric'

In [13]:
"""
import matplotlib.pyplot as plt
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label = 'train_acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0, 1, 0)
plt.legend(loc='lower right')
plt.show()
"""

"\nimport matplotlib.pyplot as plt\nx = np.arange(len(train_acc_list))\nplt.plot(x, train_acc_list, label = 'train_acc')\nplt.plot(x, test_acc_list, label='test acc', linestyle='--')\nplt.xlabel('epochs')\nplt.ylabel('accuracy')\nplt.ylim(0, 1, 0)\nplt.legend(loc='lower right')\nplt.show()\n"